In [176]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='sypark', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4048 is assigned for requested cluster
Waiting for Spark master to be available...


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38507)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Spark master launched!
Creating new Spark session, name: sypark...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [175]:
sphynx.stop()

2021년 상반기 출시 상품들 보시고, 해당 상품군의 판매 종료 시점부터 6개월 전/3개월 전/1개월 전 기간의 매출이 전체 매출에서 차지하는 비중을 정리


현재는 1년간 판매되는 상품들이 다수 존재. (자체 제작된 비 시즈널 상품의 경우 판매 기간 1년)  
해당 상품 판매 기간을 나만의 상점 아이템 등록에 맞추어 출시 이후 6개월로 제한. (콜라보등의 별도 계약건은 별도로 진행. )  
해당 시작 시점은 언제부터 진행할 것인가? (4월 노동절 아이템부터 )   
이 경우 기존 판매 상품들에 대한 매출 감소 예상.   
이를 커버하기 위해 나만의 상점 판매 건수를 현재보다 증가 시키는 것은? (현재 2달에 1번 → 매달 진행)  
이것에 대한 예측 필요.   
→ 우선은 기간 감소에 따른 매출 감소폭 확인 필요 (2020년 하반기 출시 아이템 위주) 

In [16]:
gcoin_pre = pre_load_data_mart("pc", "2021-01-01", "2021-07-28", table_name="gcoin_use")
gcoin_post = load_data_mart("pc", "2021-07-29", "2021-12-31", table_name="gcoin_use")

In [17]:
gcoin_pre = gcoin_pre.withColumn("is_salesid_exist", lit(None)).withColumn("sales_id_", lit(None))

gcoin = gcoin_pre.unionByName(gcoin_post)

In [40]:
gcoin_ = gcoin
gcoin_ = gcoin_.withColumn("month", substring(col("date"), 1, 7)).withColumn("end_month", substring(col("end_time"), 1, 7)).withColumn("start_month", substring(col("start_time"), 1, 7)).withColumn("month_diff_from_end", round(months_between(col("end_month"), col("month"))))

In [53]:
gcoin_ = gcoin_.withColumn("pass_type", \
                  when(col("sub_category") == "pc_cafe", "pc_cafe") \
                  .when(col("sub_category") == "levels_up", "survivor_pass") \
                  .when(col("sub_category") == "game", "game") \
                  .when(col("sub_category") == "bundle", "bundle") \
                  .when(col("sub_category") == "pass", "survivor_pass") \
                  .when(col("sub_category") == "tier_up", "survivor_pass") \
                  .when(col("event_type") == "esports", "esports") \
                  .when(col("event_type") == "survivorpass", "survivor_pass") \
                  .when(col("event_type") == "wsus", "wsus") \
                  .when(col("event_type") == "yourshop", "yourshop") \
                  .otherwise("ingame_items")
                 )

In [54]:
gcoin_df = gcoin_.groupBy("month", "pass_type", "month_diff_from_end", "event_name", "start_month", "end_month").agg(sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).orderBy("month", "month_diff_from_end")

In [58]:
gcoin_df.write.parquet("s3a://pubg-log-labs/seoyoung/{}/{}".format("gcoin_2021", "monthly_pc"), mode = "overwrite")

In [ ]:
gcoin_df.toPandas().to_csv("./data/gcoin_monthly.csv", index=False)

In [64]:
%%timeit
month="2021-02"
mysql.insert_table(gcoin_df.where(col("month") == month), "labs", "syp_monthly_gcoin")


KeyboardInterrupt: 

In [123]:
# gcoin_use = load_data_mart("pc", "2021-12-01", "2021-12-31", table_name="gcoin_use")

gcoin_pre = pre_load_data_mart("pc", "2020-12-01", "2020-12-31", table_name="gcoin_use")
gcoin_use = gcoin_pre.withColumn("is_salesid_exist", lit(None)).withColumn("sales_id_", lit(None))

gcoin_use = gcoin_use.withColumn("month", substring(col("date"), 1, 7)).withColumn("end_month", substring(col("end_time"), 1, 7)).withColumn("start_month", substring(col("start_time"), 1, 7)).withColumn("month_diff_from_end", round(months_between(col("end_month"), col("month"))))
gcoin_use = gcoin_use.withColumn("pass_type", \
                  when(col("sub_category") == "pc_cafe", "pc_cafe") \
                  .when(col("sub_category") == "levels_up", "survivor_pass") \
                  .when(col("sub_category") == "game", "game") \
                  .when(col("sub_category") == "bundle", "bundle") \
                  .when(col("sub_category") == "pass", "survivor_pass") \
                  .when(col("sub_category") == "tier_up", "survivor_pass") \
                  .when(col("event_type") == "esports", "esports") \
                  .when(col("event_type") == "survivorpass", "survivor_pass") \
                  .when(col("event_type") == "wsus", "wsus") \
                  .when(col("event_type") == "yourshop", "yourshop") \
                  .otherwise("ingame_items")
                 )

# gcoin_use = gcoin_.where(col("month") == "2021-04").withColumn("pass_type", \
#                   when(col("sub_category") == "pc_cafe", "pc_cafe") \
#                   .when(col("sub_category") == "levels_up", "survivor_pass") \
#                   .when(col("sub_category") == "game", "game") \
#                   .when(col("sub_category") == "bundle", "bundle") \
#                   .when(col("sub_category") == "pass", "survivor_pass") \
#                   .when(col("sub_category") == "tier_up", "survivor_pass") \
#                   .when(col("event_type") == "esports", "esports") \
#                   .when(col("event_type") == "survivorpass", "survivor_pass") \
#                   .when(col("event_type") == "wsus", "wsus") \
#                   .when(col("event_type") == "yourshop", "yourshop") \
#                   .otherwise("ingame_items")
#                  ).groupBy("month", "pass_type", "month_diff_from_end", "event_name", "start_month", "end_month").agg(sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).orderBy("month", "month_diff_from_end")

gcoin_tmp = gcoin_use.groupBy("month", "pass_type", "month_diff_from_end", "event_name", "start_month", "end_month").agg(sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).orderBy("month", "month_diff_from_end")

mysql.insert_table(gcoin_tmp, "labs", "syp_monthly_gcoin")

In [ ]:
mysql.insert_table(gcoin_df.where(col("month") == "2021-01"), "labs", "syp_monthly_gcoin")

In [57]:
mysql.drop_table("labs", "syp_monthly_gcoin")

In [ ]:
IF [sub_category]=='pc_cafe' then 'pc_cafe'

elseif [sub_category]=='levels_up' then 'survivor_pass (+levels_up)'
elseif [sub_category]=='game' then 'game'
elseif [sub_category]=='bundle' then 'bundle'
elseif [sub_category]=='pass' then 'survivor_pass (+levels_up)'
elseif [sub_category]=='tier_up' then 'survivor_pass (+levels_up)'
elseif [event_type]=='esports' then 'esport'
elseif [event_type]=='survivorpass' then 'survivor_pass (+levels_up)'
elseif [event_type]=='wsus' then 'wsus'
elseif [event_type]=='yourshop' then 'yourshop'
else 'ingame_items' END

In [ ]:
df.printSchema()

In [ ]:
df_all = gcoin_df.groupBy("month").agg(sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).withColumn("month_diff_from_end", lit(None)).withColumn("event_name", "all").withColumn("is_yourshop", lit(None)).withColumn("start_month", lit(None)).withColumn("end_month", lit(None))

In [19]:
monthly_gcoin = gcoin_.withColumn("month", substring(col("date"), 1, 7)).groupBy("month", "event_name").agg(sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).orderBy("month")

In [39]:
monthly_gcoin.toPandas().to_csv("./data/monthly_gcoin.csv", index=False)

KeyboardInterrupt: 

In [ ]:
monthly_gcoin_df = pd.read_csv("./data/monthly_gcoin.csv")

In [49]:
# mysql.insert_table(monthly_gcoin, "labs", "syp_monthly_gcoin")
mysql.drop_table("labs", "syp_monthly_gcoin")

In [22]:
gcoin = gcoin_.where((col("start_time") >= "2021-01-01") & (col("start_time") <= "2021-06-30") & (col("end_time") < "2022-03-01")).withColumn("month", substring(col("date"), 1, 7))
gcoin = gcoin.withColumn("end_month", substring(col("end_time"), 1, 7)).withColumn("start_month", substring(col("start_time"), 1, 7)).withColumn("month_diff_from_end", round(months_between(col("end_month"), col("month"))))

In [45]:
# gcoin.write.parquet("s3a://pubg-log-labs/seoyoung/f2p_dashboard/{}/{}".format("gcoin_2021", "pc_modified"), mode = "overwrite")
# gcoin = spark.read.parquet("s3a://pubg-log-labs/seoyoung/f2p_dashboard/{}/{}".format("gcoin_2021", "pc_modified"))

In [24]:
monthly_diff_gcoin = gcoin.groupBy("month", "month_diff_from_end", "event_name").agg(sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).orderBy("month", "month_diff_from_end")

In [34]:
mysql.insert_table(monthly_diff_gcoin, "labs", "syp_monthly_diff_gcoin")

In [25]:
monthly_diff_gcoin_df = monthly_diff_gcoin.toPandas()

In [27]:
monthly_diff_gcoin_df.to_csv("./data/monthly_diff_gcoin_df.csv", index=False)

In [31]:
monthly_diff_gcoin_df.sort_values(["month", "month_diff_from_end"]).head()

,month,month_diff_from_end,event_name,free_use,paid_use
0,2021-01,2.0,pgi_sales_1,50869030,427157990
1,2021-01,3.0,cn_streamers,17985660,104112950
2,2021-01,12.0,graffiti,35975910,149082210
3,2021-02,1.0,2021valentine,7487430,56111760
4,2021-02,1.0,2021lunanewyear,32370900,218318570


In [37]:
mysql.drop_table('labs', "syp_contra_cmp")

In [177]:
df = mysql.read_table(spark, "labs", "syp_monthly_event_gcoin")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38507)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38507)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [75]:
df

DataFrame[month: string, pass_type: string, month_diff_from_end: double, event_name: string, start_month: string, end_month: string, free_use: bigint, paid_use: bigint]

In [125]:
df.select("pass_type").distinct().show(truncate=False)

+-------------+
|pass_type    |
+-------------+
|ingame_items |
|bundle       |
|yourshop     |
|wsus         |
|esports      |
|survivor_pass|
+-------------+



In [132]:
df.where(col("pass_type") == "yourshop").show(truncate=False)

+-------+---------+-------------------+----------+-----------+---------+---------+---------+
|month  |pass_type|month_diff_from_end|event_name|start_month|end_month|free_use |paid_use |
+-------+---------+-------------------+----------+-----------+---------+---------+---------+
|2021-01|yourshop |9.0                |yourshop2 |2021-09    |2021-10  |500      |1880     |
|2021-03|yourshop |7.0                |yourshop2 |2021-09    |2021-10  |370      |9100     |
|2021-04|yourshop |6.0                |yourshop2 |2021-09    |2021-10  |5531     |23599    |
|2021-05|yourshop |5.0                |yourshop2 |2021-09    |2021-10  |950      |10150    |
|2021-07|yourshop |1.0                |yourshop1 |2021-07    |2021-08  |21398420 |157409560|
|2021-08|yourshop |0.0                |yourshop1 |2021-07    |2021-08  |17543400 |115891920|
|2021-09|yourshop |1.0                |yourshop2 |2021-09    |2021-10  |22503620 |64413030 |
|2021-10|yourshop |0.0                |yourshop2 |2021-09    |2021-10 

In [142]:
df.select("start_month").distinct().orderBy("start_month").show(truncate=False)

+-----------+
|start_month|
+-----------+
|null       |
|2018-11    |
|2018-12    |
|2019-01    |
|2019-06    |
|2019-08    |
|2019-09    |
|2019-11    |
|2019-12    |
|2020-06    |
|2020-07    |
|2020-08    |
|2020-09    |
|2020-10    |
|2020-11    |
|2020-12    |
|2021-01    |
|2021-02    |
|2021-03    |
|2021-04    |
+-----------+
only showing top 20 rows



pass_type in esports & ingame_items & yourshop & bundle

In [127]:
total = df.where(col("pass_type").isin(["esports", "ingame_items", "yourshop", "bundle"])).groupBy("month").agg((sum(col("paid_use") + col("free_use"))/100).alias("total_gcoin"), (sum(col("paid_use"))/100).alias("paid_gcoin")).toPandas().sort_values("month")

In [128]:
total.columns = ["month", "all_total_gcoin", "all_paid_gcoin"]

In [129]:
total.head(3)

,month,all_total_gcoin,all_paid_gcoin
9,2020-11,8468301.10,7299876.20
0,2020-12,7612321.15,6597574.90
12,2021-01,11832504.00,10257655.14


- pass_type in esports & ingame_items
- start month: 2020-11 ~ 2021-06
- month_diff_from_end is not null (case when end_month is null or event name is null)

In [150]:
monthly_month_diff = df.where((col("pass_type").isin(["esports", "ingame_items", "yourshop"])) & (~col("month_diff_from_end").isNull()) & (~col("start_month").isNull()) & (col("start_month") >= "2020-11") & (col("start_month") <= "2021-06")).groupBy("month", "pass_type", "month_diff_from_end").agg((sum(col("paid_use") + col("free_use"))/100).alias("total_gcoin"), (sum(col("paid_use"))/100).alias("paid_gcoin")).toPandas().sort_values(["month", "month_diff_from_end"])

In [151]:
monthly_month_diff.head(3)

,month,pass_type,month_diff_from_end,total_gcoin,paid_gcoin
56,2020-11,ingame_items,0.0,34861.7,31354.3
10,2020-11,ingame_items,1.0,2742854.1,2376039.4
7,2020-11,ingame_items,12.0,1603285.6,1408228.9


In [152]:
total.month = total.month.astype(str)
monthly_month_diff.month = monthly_month_diff.month.astype(str)

merged = pd.merge(monthly_month_diff, total, on="month")
merged["percentage"] = merged.total_gcoin/merged.all_total_gcoin
merged.head(3)

,month,pass_type,month_diff_from_end,total_gcoin,paid_gcoin,all_total_gcoin,all_paid_gcoin,percentage
0,2020-11,ingame_items,0.0,34861.7,31354.3,8468301.1,7299876.2,0.004117
1,2020-11,ingame_items,1.0,2742854.1,2376039.4,8468301.1,7299876.2,0.323897
2,2020-11,ingame_items,12.0,1603285.6,1408228.9,8468301.1,7299876.2,0.189328


In [153]:
merged.to_csv("./data/merged.csv", index=False)

In [159]:
by_event = df.where((col("pass_type").isin(["esports", "ingame_items", "yourshop"])) & (~col("month_diff_from_end").isNull()) & (~col("start_month").isNull()) & (col("start_month") >= "2020-11") & (col("start_month") <= "2021-06")).groupBy("month", "pass_type", "event_name", "start_month", "end_month", "month_diff_from_end").agg((sum(col("paid_use") + col("free_use"))/100).alias("total_gcoin"), (sum(col("paid_use"))/100).alias("paid_gcoin")).toPandas().sort_values(["month", "start_month"])
by_event = pd.merge(by_event, total, on="month")

In [161]:
by_event["percentage"] = by_event.total_gcoin/by_event.all_total_gcoin

In [162]:
by_event.head()

,month,pass_type,event_name,start_month,end_month,month_diff_from_end,total_gcoin,paid_gcoin,all_total_gcoin,all_paid_gcoin,percentage
0,2020-11,ingame_items,tbr,2020-11,2020-11,0.0,34861.7,31354.3,8468301.10,7299876.2,0.004117
1,2020-11,ingame_items,cn11day,2020-11,2020-12,1.0,2742854.1,2376039.4,8468301.10,7299876.2,0.323897
2,2020-11,ingame_items,DIRTBIKE,2020-11,2021-11,12.0,1603285.6,1408228.9,8468301.10,7299876.2,0.189328
3,2020-12,ingame_items,DIRTBIKE,2020-11,2021-11,11.0,704406.1,618881.5,7612321.15,6597574.9,0.092535
4,2020-12,ingame_items,cn11day,2020-11,2020-12,0.0,971345.3,842127.7,7612321.15,6597574.9,0.127602


In [163]:
by_event.to_csv("./data/by_event.csv", index=False)

In [165]:
monthly_month_diff = df.where((col("pass_type").isin(["esports", "ingame_items", "yourshop"])) & (~col("month_diff_from_end").isNull()) & (~col("start_month").isNull()) & (col("start_month") >= "2020-11") & (col("start_month") <= "2021-06")).groupBy("month", "month_diff_from_end").agg((sum(col("paid_use") + col("free_use"))/100).alias("total_gcoin"), (sum(col("paid_use"))/100).alias("paid_gcoin")).toPandas().sort_values(["month", "month_diff_from_end"])

In [170]:
tmp = pd.merge(total, monthly_month_diff, on="month")
tmp["percent"] = tmp.total_gcoin/tmp.all_total_gcoin*100
tmp.head()

,month,all_total_gcoin,all_paid_gcoin,month_diff_from_end,total_gcoin,paid_gcoin,percent
0,2020-11,8468301.10,7299876.2,0.0,34861.7,31354.3,0.411673
1,2020-11,8468301.10,7299876.2,1.0,2742854.1,2376039.4,32.389662
2,2020-11,8468301.10,7299876.2,12.0,1603285.6,1408228.9,18.932789
3,2020-12,7612321.15,6597574.9,0.0,1005948.8,865233.2,13.214745
4,2020-12,7612321.15,6597574.9,1.0,3376748.9,2971161.3,44.358992


In [178]:
all_event = df.where((col("pass_type").isin(["esports", "ingame_items", "yourshop"])) & (~col("month_diff_from_end").isNull()) & (~col("start_month").isNull())).groupBy("month", "pass_type", "event_name", "start_month", "end_month", "month_diff_from_end").agg((sum(col("paid_use") + col("free_use"))/100).alias("total_gcoin"), (sum(col("paid_use"))/100).alias("paid_gcoin")).toPandas()
all_event = pd.merge(all_event, total, on="month")
all_event.to_csv('./data/all_event.csv', index=False)